In [ ]:
CSV_PATH = "../data/cncdc_surveillance_covid19.csv"

# === 交互曲线（纯 Plotly 版，无 widgets）===
import os
import pandas as pd
import numpy as np
from datetime import date
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import HTML

# ---- 选择一个可用渲染器（避免 mimetype 错误）----
for r in ["vscode", "notebook_connected", "jupyterlab", "notebook", "colab"]:
    if r in pio.renderers:
        pio.renderers.default = r
        break
else:
    # 若上述都不可用，则在默认浏览器中打开交互图
    pio.renderers.default = "browser"

df = pd.read_csv(CSV_PATH)

# --- 时间列规范 ---
# 使用 target_end_date 作为主要时间列（监测周的结束日期）
df["report_date"] = pd.to_datetime(df.get("target_end_date"), errors="coerce")

def week_to_monday(s):
    if pd.isna(s): 
        return pd.NaT
    try:
        y, w = map(int, str(s).split("-"))
        return pd.to_datetime(date.fromisocalendar(y, w, 1))
    except Exception:
        return pd.NaT

df["week_monday"] = df.get("report_week", pd.Series([None]*len(df))).apply(week_to_monday)
df["time"] = df["report_date"].fillna(df["week_monday"])
df["time"] = pd.to_datetime(df["time"], errors="coerce")

# --- 只保留新冠（改为非捕获组，且 na=False 防止警告）---
covid_patterns = r"(?:新型冠状病毒|新冠|SARS[-\s]?CoV[-\s]?2|2019[-\s]?nCoV)"
df = df[df["pathogen"].astype(str).str.contains(covid_patterns, case=False, regex=True, na=False)].copy()

# 数值列
for c in ["ili_percent", "sari_percent"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 同一天多条 -> 取均值
df = (df.dropna(subset=["time"])
        .groupby("time", as_index=False)
        .agg({"ili_percent":"mean","sari_percent":"mean"}))
df = df.sort_values("time")

if df.empty:
    raise SystemExit("CSV 里没找到 '新型冠状病毒' 的记录，检查 pathogen 列是否名称不同。")

# --- 预计算不同平滑窗口（周）的序列 ---
windows = [1, 3, 5, 7]
series = {}
for w in windows:
    s = df.copy()
    if w > 1:
        s["ili_s"]  = s["ili_percent"].rolling(w, min_periods=1).mean()
        s["sari_s"] = s["sari_percent"].rolling(w, min_periods=1).mean()
    else:
        s["ili_s"]  = s["ili_percent"]
        s["sari_s"] = s["sari_percent"]
    series[w] = s

# --- 画多条 trace（每个窗口两条：ILI/SARI），用按钮切换可见性 ---
fig = go.Figure()
traces_per_window = 2
for w in windows:
    s = series[w]
    fig.add_trace(go.Scatter(
        x=s["time"], y=s["ili_s"], mode="lines+markers",
        name=f"ILI%（滑动{w}周）",
        hovertemplate="<b>%{x|%Y-%m-%d}</b><br>ILI: %{y:.2f}%<extra></extra>",
        visible=True if w == windows[0] else False
    ))
    fig.add_trace(go.Scatter(
        x=s["time"], y=s["sari_s"], mode="lines+markers",
        name=f"SARI%（滑动{w}周）",
        hovertemplate="<b>%{x|%Y-%m-%d}</b><br>SARI: %{y:.2f}%<extra></extra>",
        visible=True if w == windows[0] else False
    ))

# 按钮：切换平滑窗口可见性
buttons = []
for idx, w in enumerate(windows):
    vis = [False] * (len(windows) * traces_per_window)
    vis[idx*2] = True
    vis[idx*2 + 1] = True
    buttons.append(dict(label=f"{w}周", method="update", args=[{"visible": vis}]))

# 按钮：点位开关
buttons_markers = [
    dict(label="显示点", method="restyle", args=[{"mode":"lines+markers"}]),
    dict(label="不显示点", method="restyle", args=[{"mode":"lines"}]),
]

fig.update_layout(
    title="新型冠状病毒（COVID-19）监测阳性率",
    xaxis_title="日期", yaxis_title="阳性率（%）",
    hovermode="x unified", template="plotly_white", height=520,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, x=0),
    updatemenus=[
        dict(type="buttons", direction="right", x=0, y=1.15, buttons=buttons, showactive=True, pad={"r":8,"t":0}),
        dict(type="buttons", direction="right", x=0.62, y=1.15, buttons=buttons_markers, showactive=False, pad={"r":8,"t":0}),
    ]
)

# rangeslider + 常用区间按钮（注意不能用 step='week'）
fig.update_xaxes(
    type="date",
    rangeslider=dict(visible=True),
    rangeselector=dict(
        buttons=[
            dict(count=28, label="近4周",  step="day",   stepmode="backward"),
            dict(count=91, label="近13周", step="day",   stepmode="backward"),
            dict(count=6,  label="近6月",  step="month", stepmode="backward"),
            dict(step="all", label="全部"),
        ]
    )
)

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))


: 